In [2]:
#IMPORTS
########
import pandas as pd
import numpy as np
import mysql.connector
import tpclean.tpclean as tp

#custom imports
from Scripts.config import role, bucket_name, prefix, bucket_path, sub_path
from Scripts.nlp_functions import generate_wordcloud , get_summary

#establish connection
from Scripts.config import host, db
from Private.private import user , password

#NLTK
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#Imports for Bigrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from Scripts.nlp_functions import bigram_to_single_word
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#API Imports
from Private.private import aylien_app_id,aylien_API_KEY

In [3]:
print("========SETUP========")
#load credentials
conn_kwargs = {"host":host, 
               "user":user, 
               "password":password}
conn = tp.sql_connect(db,db_type="mysql",**conn_kwargs)
#conn = mysql.connector.Connect(database = db, **conn_kwargs)
c = conn.cursor()

#get all the files that dont have frequencies yet
words_df = tp.sql("""SELECT origin
FROM content
GROUP BY origin""")

i=0
try:
    job_list = list(words_df.iloc[:,0])
    print("========SETUP COMPLETE========")
except:
    print("No Jobs available")
    conn.close()
    exit()

print("========Starting JOBs========")

========SETUP========
Connecting to mysql
successfully imported module
Connection to mysql successfull. with curser MySQLCursor: (Nothing executed yet)
========SETUP COMPLETE========
========Starting JOBs========


In [6]:
file = job_list[0]

In [7]:
#VERBOSE
i += 1
print(f"Starting Job {i}/{len(job_list)}")
print(f"Current File: {file}")

#get full text
text_df = tp.sql(f"""SELECT full_text FROM conversations WHERE filename = "{file}" """)
text = text_df["full_text"][0]

#get the words from the content table
file_df = tp.sql(f"""SELECT pos_in_conv,
                LOWER(content) as content
                FROM content
                WHERE type = "pronunciation" and origin = "{file}" """)

##REMOCE STOPWORDS AND LEMMATISE
stopword_list = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

#define function for stopping and lemmatizing and apply it to the DataFrame
stoplem = lambda x: np.NaN if x in stopword_list else lemmatizer.lemmatize(x)
file_df["lemmatized"] = file_df["content"].apply(stoplem)

#create freqency dict
fDist_lemm = FreqDist(file_df["lemmatized"].dropna(),)

Starting Job 1/1
Current File: AM_101_affirmativeaction_pro.wav


In [9]:
#append frequencies to lemmatized words
freqs = pd.DataFrame.from_dict(fDist_lemm,orient = "index", columns = ["freq"])


In [25]:
#calculate Freq


#calculate freq ranks
sort_freq = freqs.sort_values(by="freq", ascending = False).reset_index()
unique_freqs = sort_freq.freq.unique()
freq_ranks = pd.DataFrame(list(zip(unique_freqs,range(len(unique_freqs)))),columns = ["freq","freq_rank"])

In [40]:
freq_ranks.head()

,freq,freq_rank
0,23,0
1,19,1
2,15,2
3,12,3
4,11,4


In [ ]:
file_df = pd.merge(file_df,freqs, how = "left", left_on = "lemmatized", right_index = True)

In [42]:
file_df = pd.merge(file_df, freq_ranks, how = "left", on = "freq")

In [ ]:
for file in job_list:
    #VERBOSE
    i += 1
    print(f"Starting Job {i}/{len(job_list)}")
    print(f"Current File: {file}")
    
    #get full text
    text_df = tp.sql(f"""SELECT full_text FROM conversations WHERE filename = "{file}" """)
    text = text_df["full_text"][0]
    
    #get the words from the content table
    file_df = tp.sql(f"""SELECT pos_in_conv,
                    LOWER(content) as content
                    FROM content
                    WHERE type = "pronunciation" and origin = "{file}" """)
    
    ##REMOCE STOPWORDS AND LEMMATISE
    stopword_list = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    #define function for stopping and lemmatizing and apply it to the DataFrame
    stoplem = lambda x: np.NaN if x in stopword_list else lemmatizer.lemmatize(x)
    file_df["lemmatized"] = file_df["content"].apply(stoplem)
    
    #create freqency dict
    fDist_lemm = FreqDist(file_df["lemmatized"].dropna(),)
    
    #append frequencies to lemmatized words
    freqs = pd.DataFrame.from_dict(fDist_lemm,orient = "index", columns = ["freq"])
    file_df = pd.merge(file_df,freqs, how = "left", left_on = "lemmatized", right_index = True)
    
    ## CREATE BIGRAMS
    finder = BigramCollocationFinder.from_words(file_df["lemmatized"].dropna())
    finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)
    bigrams_fd = finder.ngram_fd
    
    #visualize in a wordcloud
    testfd = bigram_to_single_word(bigrams_fd)
    generate_wordcloud(testfd)
    
    #SENTIMENT ANALYSIS
    sia = SentimentIntensityAnalyzer()
    sc = lambda x: sia.polarity_scores(x)["compound"]
    file_df["sentiment_score"] = file_df["lemmatized"].dropna().apply(sc)
    file_df["weight"] = file_df["freq"]*file_df["sentiment_score"]
    
    #update DB
    updates = file_df.dropna()
    for i in range(len(updates)):
        row = updates.iloc[i]
        
        #build UPDATE querry
        out =[f'{key} = "{row[key]}"' for key in row.keys()[1:]]
        set_ = ", ".join(out)
        querry = f"""UPDATE content
                     SET {set_}
                     WHERE pos_in_conv = {row['pos_in_conv']}
                     AND origin = "{file}"
                 """
        c.execute(querry)
    conn.commit()
    print("Inserted frequency dict in content")
    
    #GET SUMMARY
    summary_text = get_summary(file, text, aylien_app_id, aylien_API_KEY)
    
    c.execute(f"""UPDATE conversations
            SET summary = "{summary_text}"
            WHERE filename = "{file}" """)
    conn.commit()
    print(----------------)
conn.close()
print("========ALL JOBS FINISHED========")

In [40]:
df = tp.sql("""SELECT * FROM content
        GROUP BY origin, lemmatized
        HAVING freq IS NOT NULL
        ORDER BY origin, freq DESC""" )

In [41]:
dfs = []
for file, data in df.groupby("origin"):
    data.sort_values(by = "freq", ascending = False)
    data["freq_rank"] = data.index
    dfs.append(data)
dfs[0]

/Users/pietrassyk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pietrassyk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pietrassyk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,index,pos_in_conv,end_time,start_time,type,confidence,content,length,pause_after,origin,speaker,lemmatized,freq,sentiment_score,weight,freq_rank
0,5549,193,77.780,77.430,pronunciation,1.0000,people,0.35,0.00,AM_101_affirmativeaction_pro.wav,speaker_default,people,23.0,0.0000,0.0000,0
1,5386,30,13.300,12.850,pronunciation,1.0000,women,0.45,0.21,AM_101_affirmativeaction_pro.wav,speaker_default,woman,19.0,0.0000,0.0000,1
2,5453,97,34.950,34.630,pronunciation,1.0000,kind,0.32,0.00,AM_101_affirmativeaction_pro.wav,speaker_default,kind,15.0,0.5267,7.9005,2
3,5394,38,16.280,15.530,pronunciation,0.9331,companies,0.75,0.07,AM_101_affirmativeaction_pro.wav,speaker_default,company,12.0,0.0000,0.0000,3
4,5388,32,14.060,13.710,pronunciation,1.0000,major,0.35,0.00,AM_101_affirmativeaction_pro.wav,speaker_default,major,11.0,0.0000,0.0000,4
5,5551,195,78.350,77.900,pronunciation,1.0000,color,0.45,0.96,AM_101_affirmativeaction_pro.wav,speaker_default,color,10.0,0.0000,0.0000,5
6,5534,178,71.650,70.880,pronunciation,1.0000,particular,0.77,0.31,AM_101_affirmativeaction_pro.wav,speaker_default,particular,8.0,0.0000,0.0000,6
7,5641,285,110.670,110.490,pronunciation,0.9842,sort,0.18,0.00,AM_101_affirmativeaction_pro.wav,speaker_default,sort,8.0,0.0000,0.0000,7
8,5759,403,145.825,145.575,pronunciation,1.0000,terms,0.25,0.00,AM_101_affirmativeaction_pro.wav,speaker_default,term,8.0,0.0000,0.0000,8
9,5375,19,7.850,7.050,pronunciation,1.0000,society,0.80,0.83,AM_101_affirmativeaction_pro.wav,speaker_default,society,7.0,0.0000,0.0000,9
